# IS PROBLEMI

FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor. Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranışlardaki öbeklenmelere göre gruplar oluşturulacak.


In [1]:
! hdfs dfs -ls /user/train/datasets

Found 5 items
-rw-r--r--   1 train supergroup       4556 2020-09-23 20:56 /user/train/datasets/Advertising.csv
-rw-r--r--   1 train supergroup    7077973 2022-09-07 17:46 /user/train/datasets/airbnb_data.csv
drwxr-xr-x   - train supergroup          0 2020-11-19 21:02 /user/train/datasets/churn-telecom
-rw-r--r--   1 train supergroup   19422619 2022-09-28 08:34 /user/train/datasets/flo100k_data.csv
drwxr-xr-x   - train supergroup          0 2020-11-21 11:16 /user/train/datasets/retail_db


In [2]:
! hdfs dfs -put /home/train/datasets/flo100k_data.csv/ /user/train/datasets

In [9]:
!ls "/home/train/datasets"

Advertising.csv  churn-telecom	   iris.csv
airbnb_data.csv  flo100k_data.csv  retail_db


## Import Libraries

In [2]:
import findspark

In [3]:
findspark.init("/opt/manual/spark")

In [4]:
from pyspark.sql import SparkSession, functions as F

In [5]:
import pandas as pd

In [8]:
spark= (
SparkSession.builder
    .appName("Flo Customer Segmentation")
    .master("yarn")
    .config("spark.sql.shuffle.partitions","2")
    .getOrCreate()
)

## Read Data

In [9]:
# Not:Veriyi infer Schema True olacak sekilde okutunuz, ilgili csv dosyasına göre delimiter seçiniz

df =(spark.read.format("csv")
     .option("header", True)
     .option("delimiter", "|")
     .option("inferSchema", True)
     .load("/user/train/datasets/flo100k_data.csv"))

In [8]:
#df.persist()

DataFrame[|master_id|order_channel|platform_type|last_order_channel|first_order_date|last_order_date|last_order_date_online|last_order_date_offline|order_num_total_ever_online|order_num_total_ever_offline|customer_value_total_ever_offline|customer_value_total_ever_online|interested_in_categories_12|last_order_date_new: string]

In [9]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [10]:
#İlk 5 gözlemini
df.limit(5).toPandas()

,_c0,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,last_order_date_new
0,0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,212.98,0.00,None,2019-02-23
1,1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,NaN,1.0,199.98,0.00,None,2019-12-01
2,2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,NaN,1.0,140.49,0.00,[ERKEK],2020-07-24
3,3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,NaN,0.00,174.99,None,2018-12-31
4,4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,NaN,0.00,283.95,[],2021-05-05


In [11]:
# Toplam gözlem sayısını
df_count =df.count()
print(df_count)

100000


In [12]:
# Toplam değişken sayısını
print(len(df.columns))

15


In [13]:
# Değişken tiplerini inceleyiniz
df.dtypes

[('_c0', 'int'),
 ('master_id', 'string'),
 ('order_channel', 'string'),
 ('platform_type', 'string'),
 ('last_order_channel', 'string'),
 ('first_order_date', 'timestamp'),
 ('last_order_date', 'timestamp'),
 ('last_order_date_online', 'timestamp'),
 ('last_order_date_offline', 'timestamp'),
 ('order_num_total_ever_online', 'double'),
 ('order_num_total_ever_offline', 'double'),
 ('customer_value_total_ever_offline', 'double'),
 ('customer_value_total_ever_online', 'double'),
 ('interested_in_categories_12', 'string'),
 ('last_order_date_new', 'string')]

### Adım 3: Eksik Gözlemler

- Veri setindekideğişkenlerineksikgözlemleriniinceleyiniz

In [14]:
def null_count(df, col_name):
    nc = df.select(col_name).filter(
    (F.col(col_name) == "NA") |
    (F.col(col_name) == "" )|
    (F.col(col_name).isNull())
    ).count()
    
    return nc

In [17]:
for col_name in df.dtypes:
    nc = null_count(df, col_name[0])
    if nc > 0:
        print("{} has {} % {} missing values".format(col_name[0], nc, (nc/df_count) * 100))
        

last_order_date_online has 70784 % 70.784 missing values
last_order_date_offline has 21703 % 21.703 missing values
order_num_total_ever_online has 70784 % 70.784 missing values
order_num_total_ever_offline has 21703 % 21.703 missing values
interested_in_categories_12 has 56590 % 56.589999999999996 missing values


## Veri analizi

Adım 1: UniqueDeğişken Kontrolü
- master_id değişkeninin unique olup olmadığını kontrol ediniz

In [20]:
df.select("master_id").distinct().count()

100000

## Adım 2: Veriyi Anlama –Groupby

- platform_type & order_channel kırılımında gözlem sayılarına bakınız

In [19]:
df.groupby(['platform_type', 'order_channel']).count().show()

+-------------+-------------+-----+
|platform_type|order_channel|count|
+-------------+-------------+-----+
|  OmniChannel|      Offline| 4793|
|       Online|      Ios App| 3008|
|  OmniChannel|      Desktop| 1498|
|      Offline|      Offline|65991|
|       Online|  Android App| 8728|
|       Online|      Desktop| 3253|
|  OmniChannel|       Mobile| 2061|
|       Online|       Mobile| 6451|
|  OmniChannel|  Android App| 3261|
|  OmniChannel|      Ios App|  956|
+-------------+-------------+-----+



## Adım 3: Omnichannel

Omnichannel müşterilerin hem online hem de offline platformlardan alışveriş yaptığını ifade etmektedir.

Herbir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

- toplam alışveriş sayısı için değişken ismini order_num_total yapınız.
- toplam harcama için değişken ismini customer_value_total yapınız

**Not:** Yeni değişkenleri oluşturmadan önce bu değişkenlerin null değerlerini sıfıra ceviriniz ve tüm değerlerinin sıfıra eşit veya büyük olduğuna emin olunuz.

In [ ]:
order_num_total_ever_online
order_num_total_ever_offline

In [23]:
#change nan values to 0

df = df.na.fill(value=0, 
         subset= ['order_num_total_ever_online', 'order_num_total_ever_offline', 'customer_value_total_ever_online',
                 'customer_value_total_ever_offline'])

In [28]:
#yukaridaki degisiklikten emmin olmak icin
df = df.filter((df.order_num_total_ever_online >= 0) &
          (df.order_num_total_ever_offline >= 0) &
          (df.customer_value_total_ever_online >= 0) &
          (df.customer_value_total_ever_offline >= 0)
         )

In [29]:
df.count()

100000

In [31]:
df = df.withColumn("order_num_total",
              df.order_num_total_ever_online + df.order_num_total_ever_offline)

In [32]:
df = df.withColumn("customer_value_total",
                  df.customer_value_total_ever_online + df.customer_value_total_ever_offline)

In [33]:
df.limit(5).toPandas()

,_c0,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,last_order_date_new,order_num_total,customer_value_total
0,0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,0.0,1.0,212.98,0.00,None,2019-02-23,1.0,212.98
1,1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,0.0,1.0,199.98,0.00,None,2019-12-01,1.0,199.98
2,2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,0.0,1.0,140.49,0.00,[ERKEK],2020-07-24,1.0,140.49
3,3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,0.0,0.00,174.99,None,2018-12-31,1.0,174.99
4,4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,0.0,0.00,283.95,[],2021-05-05,2.0,283.95


## Görev 2: Veri Analizi

**Adim 4:** Kanal Incelemesi

In [41]:
#Kanallardaki toplam müsteri, ortalama alinan ürün ve kazanc dagilimina bakiniz. order_channel

df.groupby("order_channel").agg({"master_id":"count",
                                "order_num_total":"mean",
                                "customer_value_total":"mean"}).show()

#avg("order_num_total", "customer_value_total").show()


+-------------+----------------+-------------------------+--------------------+
|order_channel|count(master_id)|avg(customer_value_total)|avg(order_num_total)|
+-------------+----------------+-------------------------+--------------------+
|      Offline|           70784|        218.1639484912975|  1.6003475361663653|
|      Ios App|            3964|        568.4640312815283|   3.377648839556004|
|       Mobile|            8512|        391.7418761748011|   2.798637218045113|
|      Desktop|            4751|       376.91355504103785|   2.538623447695222|
|  Android App|           11989|        532.8462840937639|    3.50971724080407|
+-------------+----------------+-------------------------+--------------------+



**Adim 5:** Platforma göre satis degerleri

In [44]:
#Platformlardaki toplam müsteri, ortalama alinan ürün ve kazanc dagilimina bakiniz.

df.groupby("platform_type").agg({"master_id":"count",
                                "order_num_total":"mean",
                                "customer_value_total":"mean"}).show()



#agg(count("master_id").alias("total_customers"),
                               #avg("order_num_total").alias("average orders"),
                                #avg("customer_value_total").alias("average gain")
                                   #).show()

+-------------+----------------+-------------------------+--------------------+
|platform_type|count(master_id)|avg(customer_value_total)|avg(order_num_total)|
+-------------+----------------+-------------------------+--------------------+
|      Offline|           65991|       215.73030686012984|  1.5837917291751906|
|  OmniChannel|           12569|        500.7937512928769|  3.4947092051873656|
|       Online|           21440|       404.78965764929046|  2.6207089552238805|
+-------------+----------------+-------------------------+--------------------+



## Görev 3:  K-MeansMetriklerinin Hesaplanmasi

**Adim 1:** RFM 

- master_id, last_order_date_new, order_num_total, customer_value_total değişkenlerinden rfm adında yeni bir dataframe olusturunuz

In [45]:
rfm = df.select("master_id", "last_order_date_new", "order_num_total", "customer_value_total")

In [46]:
rfm.limit(5).toPandas()

,master_id,last_order_date_new,order_num_total,customer_value_total
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,2019-02-23,1.0,212.98
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,2019-12-01,1.0,199.98
2,602897a6-cdac-11ea-b31f-000d3a38a36f,2020-07-24,1.0,140.49
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,2018-12-31,1.0,174.99
4,80664354-adf0-11eb-8f64-000d3a299ebf,2021-05-05,2.0,283.95


**Adim 2:** Son satin Alma Tarihi
- last_order_date degiskeni üzerinden son satin alma tarihini bulunuz.

In [57]:
#collect önemli!!!, bir veri almak icin collect'i kullanabiliriz!
latest_date = df.agg({"last_order_date": "max"}).collect()[0][0]
latest_date.date()

datetime.date(2021, 5, 30)

**Adim 3:** Recency

Son satim alim tarihinden iki gün sonrasi üzerinden, her müsterinin son satin alma sürelerini gün cinsinden hesaplayip, Recency adinda bir degiskende kaydediniz.

In [63]:
from pyspark.sql.functions import expr

rfm = rfm.withColumn("Recency", expr("datediff('2021-6-1', last_order_date_new)"))
rfm.show(5)

+--------------------+-------------------+---------------+--------------------+-------+
|           master_id|last_order_date_new|order_num_total|customer_value_total|Recency|
+--------------------+-------------------+---------------+--------------------+-------+
|b3ace094-a17f-11e...|         2019-02-23|            1.0|              212.98|    829|
|c57d7c4c-a950-11e...|         2019-12-01|            1.0|              199.98|    548|
|602897a6-cdac-11e...|         2020-07-24|            1.0|              140.49|    312|
|388e4c4e-af86-11e...|         2018-12-31|            1.0|              174.99|    883|
|80664354-adf0-11e...|         2021-05-05|            2.0|              283.95|     27|
+--------------------+-------------------+---------------+--------------------+-------+
only showing top 5 rows



**Adim 4:** Yeniden Isimlendirme

- order_num_total degiskenini Frequency
- customer_value_total degiskenini Monetary adinda degistiriniz

In [64]:
rfm = (rfm.withColumnRenamed("order_num_total", "Frequency")
      .withColumnRenamed("customer_value_total", "Monetary"))

rfm.show(5)

+--------------------+-------------------+---------+--------+-------+
|           master_id|last_order_date_new|Frequency|Monetary|Recency|
+--------------------+-------------------+---------+--------+-------+
|b3ace094-a17f-11e...|         2019-02-23|      1.0|  212.98|    829|
|c57d7c4c-a950-11e...|         2019-12-01|      1.0|  199.98|    548|
|602897a6-cdac-11e...|         2020-07-24|      1.0|  140.49|    312|
|388e4c4e-af86-11e...|         2018-12-31|      1.0|  174.99|    883|
|80664354-adf0-11e...|         2021-05-05|      2.0|  283.95|     27|
+--------------------+-------------------+---------+--------+-------+
only showing top 5 rows



**Adim 5:** Filtre

- last_order_date_new degiskenini rfm dataframe'inden cikariniz

In [65]:
rfm = rfm.drop("last_order_date_new")

In [66]:
rfm.show(5)

+--------------------+---------+--------+-------+
|           master_id|Frequency|Monetary|Recency|
+--------------------+---------+--------+-------+
|b3ace094-a17f-11e...|      1.0|  212.98|    829|
|c57d7c4c-a950-11e...|      1.0|  199.98|    548|
|602897a6-cdac-11e...|      1.0|  140.49|    312|
|388e4c4e-af86-11e...|      1.0|  174.99|    883|
|80664354-adf0-11e...|      2.0|  283.95|     27|
+--------------------+---------+--------+-------+
only showing top 5 rows



## Görev 4: Modele Hazirlik

**Adim 1:** Vector Assembler

- Recency
- Frequency
- Monetary

degiskenlerine Vector Assembler uygulayiniz

In [71]:
from pyspark.ml.feature import VectorAssembler

In [75]:
rfm_col = ["Recency", "Frequency", "Monetary"]

In [76]:
assembler = (VectorAssembler()
             .setHandleInvalid("skip")
             .setInputCols(rfm_col).
             setOutputCol("unscaled_features"))

**Adim 2:** Standart Scaler

- Vector Assembler uyguladiginiz degiskenlere Standart Scaler islemini uygulayiniz.

In [80]:
from pyspark.ml.feature import StandardScaler

In [81]:
scaler = StandardScaler().setInputCol("unscaled_features").setOutputCol("features")

**Adim 3:** Pipeline
- rfm dataframeini kullanarak bir pipeline olusturunuz

In [82]:
from pyspark.ml import Pipeline

In [83]:
pipeline_obj = Pipeline().setStages([assembler, scaler])

In [84]:
pipeline_model = pipeline_obj.fit(rfm) #ögren

transformed_df = pipeline_model.transform(rfm) #uygula

transformed_df.show(5)

+--------------------+---------+--------+-------+------------------+--------------------+
|           master_id|Frequency|Monetary|Recency| unscaled_features|            features|
+--------------------+---------+--------+-------+------------------+--------------------+
|b3ace094-a17f-11e...|      1.0|  212.98|    829|[829.0,1.0,212.98]|[3.24793811935259...|
|c57d7c4c-a950-11e...|      1.0|  199.98|    548|[548.0,1.0,199.98]|[2.14700855175539...|
|602897a6-cdac-11e...|      1.0|  140.49|    312|[312.0,1.0,140.49]|[1.22238443092643...|
|388e4c4e-af86-11e...|      1.0|  174.99|    883|[883.0,1.0,174.99]|[3.45950465547448...|
|80664354-adf0-11e...|      2.0|  283.95|     27| [27.0,2.0,283.95]|[0.10578326806094...|
+--------------------+---------+--------+-------+------------------+--------------------+
only showing top 5 rows



## Görev 5: K-Means

**Adim 1:** Optimum Küme Sayisi

KMeans küme sayisi ve hatalarini hesaplayip, bir küme sayisi seciniz

In [85]:
from pyspark.ml.clustering import KMeans

In [87]:
def compute_kmeans_model(df, k):
    kmeansObject = KMeans() \
    .setSeed(142) \
    .setK(k)
    
    return kmeansObject.fit(df)

## evaluate Model

In [88]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [89]:
evaluator = ClusteringEvaluator()

In [90]:
score_list = []

In [92]:
for k in range(2,11):
    
    kmeans_model = compute_kmeans_model(transformed_df, k)
    
    evaluated_df = kmeans_model.transform(transformed_df)
    
    score = evaluator.evaluate(evaluated_df)
    score_list.append(score)
    
    
    print("k: {}, score: {}".format(k, score))

k: 2, score: 0.440326030862553
k: 3, score: 0.6549567983919242
k: 4, score: 0.6705380055471301
k: 5, score: 0.5063361815252156
k: 6, score: 0.512413557879143
k: 7, score: 0.5471947591534062
k: 8, score: 0.5710920011281072
k: 9, score: 0.5310175131579817
k: 10, score: 0.5571794995718167


## Final Model

- Sectiginiz Küme Sayisi ile yeni modeli kurunuz

In [93]:
#Pyspark siniflandirma icin Silhouette modelini kullaniyor, 1'e yakin olan küme sayisi tercih edilebilir

kmeans = (KMeans() \
         .setSeed(142) \
         .setK(4))

In [94]:
kmeans_model = kmeans.fit(transformed_df)

In [95]:
transformed = kmeans_model.transform(transformed_df)

In [96]:
transformed.show()

+--------------------+---------+------------------+-------+--------------------+--------------------+----------+
|           master_id|Frequency|          Monetary|Recency|   unscaled_features|            features|prediction|
+--------------------+---------+------------------+-------+--------------------+--------------------+----------+
|b3ace094-a17f-11e...|      1.0|            212.98|    829|  [829.0,1.0,212.98]|[3.24793811935259...|         3|
|c57d7c4c-a950-11e...|      1.0|            199.98|    548|  [548.0,1.0,199.98]|[2.14700855175539...|         3|
|602897a6-cdac-11e...|      1.0|            140.49|    312|  [312.0,1.0,140.49]|[1.22238443092643...|         2|
|388e4c4e-af86-11e...|      1.0|            174.99|    883|  [883.0,1.0,174.99]|[3.45950465547448...|         3|
|80664354-adf0-11e...|      2.0|            283.95|     27|   [27.0,2.0,283.95]|[0.10578326806094...|         2|
|47511f36-aeb4-11e...|      1.0|            139.98|    933|  [933.0,1.0,139.98]|[3.6553995963280

# Adım 3: Tanımlayıcı istatistikler

- Oluşturduğunuz kümelerin tanımlayıcı istatistiklerini inceleyiniz.


In [98]:
from pyspark.sql.functions import sum, col, desc, mean, count

transformed.groupby("prediction") \
.agg(count("Monetary"). alias("count"), \
    mean("Monetary"). alias("avg_monetary"), \
    mean("Recency").alias("avg_recency"), \
    mean("Frequency").alias("avg_frequency")) \
.sort(desc("avg_monetary")) \
.show()

+----------+-----+------------------+------------------+------------------+
|prediction|count|      avg_monetary|       avg_recency|     avg_frequency|
+----------+-----+------------------+------------------+------------------+
|         1|    6| 31003.60666666667|             343.5|226.83333333333334|
|         0| 3493|1677.1947953048953|191.19982822788435|10.410821643286573|
|         2|42303| 317.2782289672691|173.50268302484457| 2.090466397182233|
|         3|54198|179.76939407365893| 615.3469131702277| 1.447839403668032|
+----------+-----+------------------+------------------+------------------+

